In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import requests
import geocoder
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

**NOTE: The API keys have been removed from this notebook before being committed to Github.  All of the results from here on were run prior to this removal, so should be fine, but if you do a Kernel > Restart & Run All without inserting new API keys, much of Section Three will be errors!**

# Section One

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
toronto_neighborhoods = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(toronto_neighborhoods, 'lxml')

In [3]:
data = []
headers = []
table = soup.find('table', class_='wikitable sortable').tbody

rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('th') + row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if row.find_all('th') > row.find_all('td'):
        headers.append([ele for ele in cols if ele])
    else:
        data.append([ele for ele in cols if ele])

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
dataframe = pd.DataFrame(data, columns = headers[0])
# Rename the first & third columns
dataframe.rename(columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'}, inplace=True)
dataframe

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df_all_boroughs = dataframe[dataframe.Borough != 'Not assigned']
df_all_boroughs.reset_index(inplace=True, drop=True)
df_all_boroughs

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
df_all_boroughs_corrected = df_all_boroughs.copy()
df_all_boroughs_corrected.loc[df_all_boroughs_corrected['Neighborhood'] == 'Not assigned',
                    'Neighborhood'] = df_all_boroughs_corrected['Borough']
df_all_boroughs_corrected.reset_index(inplace=True, drop=True)
df_all_boroughs_corrected

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
# Output will be a series, so need to convert back to df.
df_ab_grouped = pd.DataFrame(df_all_boroughs_corrected.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join))
df_ab_grouped.reset_index(inplace=True)
df_ab_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df_ab_grouped.shape

(103, 3)

# Section Two

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [9]:
# Get latitudes and longitudes for all neighborhoods from Arcgis
toronto_neighborhood_lat_lng = {}

rows = df_ab_grouped['PostalCode']
for row in rows:
    lat_lng = geocoder.arcgis('{}, Toronto, Ontario'.format(row)).latlng
    
    # key = PostalCode, lat = [PostalCode][0], long = [PostalCode][1]
    toronto_neighborhood_lat_lng[row] = lat_lng

In [10]:
# Dict with all neighborhood latidudes and longitudes
toronto_neighborhood_lat_lng

{'M1B': [43.811525000000074, -79.19551721399995],
 'M1C': [43.78573000000006, -79.15874999999994],
 'M1E': [43.76569000000006, -79.17525603599995],
 'M1G': [43.76835912100006, -79.21758999999997],
 'M1H': [43.76968799900004, -79.23943999999995],
 'M1J': [43.74312500000008, -79.23174973599998],
 'M1K': [43.726244585000074, -79.26366999999993],
 'M1L': [43.71313321100007, -79.28505499999994],
 'M1M': [43.72357500000004, -79.23497617799995],
 'M1N': [43.69666500000005, -79.26016331599999],
 'M1P': [43.759975000000054, -79.26897402899993],
 'M1R': [43.750710464000065, -79.30055999999996],
 'M1S': [43.79394000000008, -79.26798280099996],
 'M1T': [43.78472500000004, -79.29904659999994],
 'M1V': [43.817810000000065, -79.28024362199994],
 'M1W': [43.80088094900003, -79.32073999999994],
 'M1X': [43.83421500000003, -79.21670085099998],
 'M2H': [43.80284500000005, -79.35623615099996],
 'M2J': [43.780880000000025, -79.34779577599994],
 'M2K': [43.781015000000025, -79.38054242199996],
 'M2L': [43.7

In [11]:
# Append latitudes & longitudes to dataframe
df_ab_grouped_latlng = df_ab_grouped.copy()

df_ab_grouped_latlng['Latitude'] = \
    df_ab_grouped_latlng['PostalCode'].map(pd.DataFrame.from_dict(toronto_neighborhood_lat_lng).transpose()[0])
df_ab_grouped_latlng['Longitude'] = \
    df_ab_grouped_latlng['PostalCode'].map(pd.DataFrame.from_dict(toronto_neighborhood_lat_lng).transpose()[1])

df_ab_grouped_latlng

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163


# Section Three

#### Create a map of Toronto and its neighborhoods

In [12]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for postcode, lat, lng, borough, neighborhood in zip(df_ab_grouped_latlng['PostalCode'], df_ab_grouped_latlng['Latitude'], df_ab_grouped_latlng['Longitude'], df_ab_grouped_latlng['Borough'], df_ab_grouped_latlng['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhood, borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

**NOTE: The API keys have been removed from this notebook before being committed to Github.  All of the results from here on were run prior to this removal, so should be fine, but if you do a Kernel > Restart & Run All without inserting new API keys, much of Section Three will be errors!**

In [41]:
# CLIENT_ID & CLIENT_SECRET REDACTED BEFORE COMMIT TO GITHUB

CLIENT_ID = 'CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLIENT_ID
CLIENT_SECRET:CLIENT_SECRET


#### Let's explore the first neighborhood in our dataframe.

In [15]:
neighborhood_latitude = df_ab_grouped_latlng.loc[0, 'Latitude']
neighborhood_longitude = df_ab_grouped_latlng.loc[0, 'Longitude']
neighborhood_name = df_ab_grouped_latlng.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.811525000000074, -79.19551721399995.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [42]:
url = 'https://api.foursquare.com/v2/venues/search?' +\
    'll=' + str(neighborhood_latitude) + ',' + str(neighborhood_longitude) +\
    '&radius=500' +\
    '&client_id=' + CLIENT_ID +\
    '&client_secret=' + CLIENT_SECRET +\
    '&v=' + str(VERSION)
print(url)

https://api.foursquare.com/v2/venues/search?ll=43.811525000000074,-79.19551721399995&radius=500&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=20180605


In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d162c6cad1789002ca82dee'},
 'response': {'venues': [{'id': '561aa40b498e1bc38c11111c',
    'name': 'Alvin Curling Public School',
    'location': {'lat': 43.808683,
     'lng': -79.190103,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.808683,
       'lng': -79.190103}],
     'distance': 537,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4f4533804b9074f6e4fb0105',
      'name': 'Elementary School',
      'pluralName': 'Elementary Schools',
      'shortName': 'Elementary School',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1561734252',
    'hasPerk': False},
   {'id': '4d99e344942ba093b75aa58c',
    'name': 'Enterprise Holdings Inc',
    'location': {'address': '709 Milner Ave.',
     'lat': 43.795848698256805,
     'lng': -79.21355937929297,
     'labeledLatLngs'

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            10)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=df_ab_grouped_latlng['Neighborhood'],
                                   latitudes=df_ab_grouped_latlng['Latitude'],
                                   longitudes=df_ab_grouped_latlng['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(706, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.765690,-79.175256,The Strawberry Patch,43.764738,-79.173081,Tea Room
4,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",10,10,10,10,10,10
Agincourt,10,10,10,10,10,10
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",1,1,1,1,1,1
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",5,5,5,5,5,5
"Bathurst Manor,Downsview North,Wilson Heights",1,1,1,1,1,1
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 178 unique categories.


## 3. Analyze Each Neighborhood

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.1,0.0,0.0
1,Agincourt,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
6,Bayview Village,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.25,0.00,0.0,0.0,0.0
7,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.1,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
8,Berczy Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.1,0.0,0.0,0.0,0.0,0.00,0.00,0.1,0.0,0.0
9,"Birch Cliff,Cliffside West",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                           venue  freq
0             Seafood Restaurant   0.1
1            American Restaurant   0.1
2  Vegetarian / Vegan Restaurant   0.1
3           Gym / Fitness Center   0.1
4                            Gym   0.1


----Agincourt----
                 venue  freq
0        Shopping Mall   0.2
1                 Park   0.1
2                 Pool   0.1
3  Shanghai Restaurant   0.1
4          Supermarket   0.1


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                      venue  freq
0                  Pharmacy   1.0
1                      Park   0.0
2            Massage Studio   0.0
3  Mediterranean Restaurant   0.0
4               Men's Store   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
            venue  freq
0    Liquor Store   0.1
1  Hardware Store   0.1
2        Pharmacy   0.1
3     Pizza Place   0.1
4   Grocery Store   0.1


----Alderwood,

                      venue  freq
0                      Park  0.25
1                    Bakery  0.25
2          Basketball Court  0.25
3             Deli / Bodega  0.25
4  Mediterranean Restaurant  0.00


----East Birchmount Park,Ionview,Kennedy Park----
              venue  freq
0  Department Store  0.25
1       Coffee Shop  0.25
2        Playground  0.25
3    Discount Store  0.25
4     Movie Theater  0.00


----East Toronto----
                venue  freq
0                Park  0.25
1  Italian Restaurant  0.25
2                 Bar  0.25
3      Farmers Market  0.25
4     Organic Grocery  0.00


----Emery,Humberlea----
                      venue  freq
0               Coffee Shop  0.50
1                      Park  0.25
2                 Nightclub  0.25
3             Movie Theater  0.00
4  Mediterranean Restaurant  0.00


----Fairview,Henry Farm,Oriole----
               venue  freq
0             Bakery   0.1
1            Theater   0.1
2       Burger Joint   0.1
3  Electronics Store  

                venue  freq
0                Café   0.2
1         Flower Shop   0.1
2  Italian Restaurant   0.1
3  Frozen Yogurt Shop   0.1
4           Gastropub   0.1


----Ryerson,Garden District----
           venue  freq
0    Pizza Place   0.1
1           Café   0.1
2     Comic Shop   0.1
3  Burrito Place   0.1
4          Plaza   0.1


----Scarborough Village----
                        venue  freq
0           Indian Restaurant  0.25
1               Train Station  0.25
2               Grocery Store  0.25
3                  Restaurant  0.25
4  Modern European Restaurant  0.00


----Silver Hills,York Mills----
                        venue  freq
0                 Music Venue   1.0
1                 Yoga Studio   0.0
2  Modern European Restaurant   0.0
3              Massage Studio   0.0
4    Mediterranean Restaurant   0.0


----St. James Town----
            venue  freq
0       Gastropub   0.2
1     Coffee Shop   0.2
2  Cosmetics Shop   0.1
3        Creperie   0.1
4      Food Truck  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Gym / Fitness Center,Steakhouse,Vegetarian / Vegan Restaurant,Coffee Shop,American Restaurant,Gym,Restaurant,Gastropub,Café,Seafood Restaurant
1,Agincourt,Shopping Mall,Sushi Restaurant,Bakery,Shanghai Restaurant,Chinese Restaurant,Park,Badminton Court,Pool,Supermarket,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner,Dessert Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Liquor Store,Sandwich Place,Grocery Store,Hardware Store,Pharmacy,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Japanese Restaurant
4,"Alderwood,Long Branch",Gym,Convenience Store,Sandwich Place,Pub,Dance Studio,Construction & Landscaping,Dessert Shop,Eastern European Restaurant,Dog Run,Doctor's Office


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [28]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 0, 1, 3, 0, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Gym / Fitness Center,Steakhouse,Vegetarian / Vegan Restaurant,Coffee Shop,American Restaurant,Gym,Restaurant,Gastropub,Café,Seafood Restaurant
1,Agincourt,Shopping Mall,Sushi Restaurant,Bakery,Shanghai Restaurant,Chinese Restaurant,Park,Badminton Court,Pool,Supermarket,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner,Dessert Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Liquor Store,Sandwich Place,Grocery Store,Hardware Store,Pharmacy,Pizza Place,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Japanese Restaurant
4,"Alderwood,Long Branch",Gym,Convenience Store,Sandwich Place,Pub,Dance Studio,Construction & Landscaping,Dessert Shop,Eastern European Restaurant,Dog Run,Doctor's Office
5,"Bathurst Manor,Downsview North,Wilson Heights",Men's Store,Women's Store,College Stadium,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner
6,Bayview Village,Park,Trail,Dog Run,Construction & Landscaping,Deli / Bodega,Electronics Store,Eastern European Restaurant,Doctor's Office,Discount Store,Diner
7,"Bedford Park,Lawrence Manor East",Greek Restaurant,Thai Restaurant,Comfort Food Restaurant,Café,Pub,Restaurant,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Coffee Shop
8,Berczy Park,Vegetarian / Vegan Restaurant,Cocktail Bar,Breakfast Spot,Park,Museum,Concert Hall,Liquor Store,Thai Restaurant,Farmers Market,Steakhouse
9,"Birch Cliff,Cliffside West",Gym Pool,General Entertainment,Park,Skating Rink,College Stadium,Gym,Dance Studio,Eastern European Restaurant,Dog Run,Doctor's Office


In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pd.DataFrame(df_ab_grouped_latlng).copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,4.0,Home Service,Women's Store,Department Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,1.0,Construction & Landscaping,Bar,Women's Store,Dessert Shop,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256,3.0,Park,Tea Room,Construction & Landscaping,Gym / Fitness Center,Gourmet Shop,Electronics Store,Dog Run,Doctor's Office,Discount Store,Diner
3,M1G,Scarborough,Woburn,43.768359,-79.217590,3.0,Park,Coffee Shop,Korean Restaurant,Business Service,Department Store,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1.0,Lounge,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner,Dessert Shop
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750,0.0,Grocery Store,Train Station,Restaurant,Indian Restaurant,Women's Store,Deli / Bodega,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670,1.0,Coffee Shop,Playground,Discount Store,Department Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Diner
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055,1.0,Coffee Shop,Intersection,Bus Line,Bus Station,Soccer Field,Metro Station,Bakery,Women's Store,Electronics Store,Eastern European Restaurant
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976,1.0,Fast Food Restaurant,Coffee Shop,Furniture / Home Store,Sandwich Place,Discount Store,Burger Joint,Liquor Store,Pharmacy,Pizza Place,Construction & Landscaping
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163,3.0,Gym Pool,General Entertainment,Park,Skating Rink,College Stadium,Gym,Dance Studio,Eastern European Restaurant,Dog Run,Doctor's Office


Check for NaN's in Foursquare results

In [35]:
foursquare_nan = toronto_merged[toronto_merged.isna().any(axis=1)]
foursquare_nan

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.834215,-79.216701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,M5N,Central Toronto,Roselawn,43.711941,-79.419120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Drop the two neighborhoods Foursquare didn't have results for

In [36]:
toronto_merged_foursquare_only = toronto_merged.dropna()
toronto_merged_foursquare_only

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,4.0,Home Service,Women's Store,Department Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,1.0,Construction & Landscaping,Bar,Women's Store,Dessert Shop,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256,3.0,Park,Tea Room,Construction & Landscaping,Gym / Fitness Center,Gourmet Shop,Electronics Store,Dog Run,Doctor's Office,Discount Store,Diner
3,M1G,Scarborough,Woburn,43.768359,-79.217590,3.0,Park,Coffee Shop,Korean Restaurant,Business Service,Department Store,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1.0,Lounge,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store,Diner,Dessert Shop
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750,0.0,Grocery Store,Train Station,Restaurant,Indian Restaurant,Women's Store,Deli / Bodega,Eastern European Restaurant,Dog Run,Doctor's Office,Discount Store
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670,1.0,Coffee Shop,Playground,Discount Store,Department Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dog Run,Doctor's Office,Diner
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055,1.0,Coffee Shop,Intersection,Bus Line,Bus Station,Soccer Field,Metro Station,Bakery,Women's Store,Electronics Store,Eastern European Restaurant
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976,1.0,Fast Food Restaurant,Coffee Shop,Furniture / Home Store,Sandwich Place,Discount Store,Burger Joint,Liquor Store,Pharmacy,Pizza Place,Construction & Landscaping
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163,3.0,Gym Pool,General Entertainment,Park,Skating Rink,College Stadium,Gym,Dance Studio,Eastern European Restaurant,Dog Run,Doctor's Office


Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_foursquare_only['Latitude'], toronto_merged_foursquare_only['Longitude'], toronto_merged_foursquare_only['Neighborhood'], toronto_merged_foursquare_only['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters